## Convolution Layer Example

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()

    ## 조건1: conv layer 2개 & fc layer 2개
    ## 조건2: kernel size=5, stride=1로 통일
    ## output을 참고하여 차원을 계산해주세요!
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=5, stride=1)
    self.conv2 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=5, stride=1)
    self.fc1 = nn.Linear(in_features=10*12*12, out_features=50)
    self.fc2 = nn.Linear(in_features=50, out_features=10)

  def forward(self, x):
    print("연산 전", x.size())
    x = F.relu(self.conv1(x))
    print("conv1 연산 후", x.size())
    x = F.relu(self.conv2(x))
    print("conv2 연산 후",x.size())
    x = x.view(-1, 10 * 12 * 12)
    print("차원 감소 후", x.size())
    x = F.relu(self.fc1(x))
    print("fc1 연산 후", x.size())
    x = self.fc2(x)
    print("fc2 연산 후", x.size())
    return x

cnn = CNN()
output = cnn(torch.randn(10, 1, 20, 20))  # Input Size: (10, 1, 20, 20)

연산 전 torch.Size([10, 1, 20, 20])
conv1 연산 후 torch.Size([10, 3, 16, 16])
conv2 연산 후 torch.Size([10, 10, 12, 12])
차원 감소 후 torch.Size([10, 1440])
fc1 연산 후 torch.Size([10, 50])
fc2 연산 후 torch.Size([10, 10])


## Max Pooling Layer Example

In [6]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()

    ## 조건1: 최대풀링 layer 2개 & fc layer 2개
    ## 조건2: kernel size=2로 통일, stride는 따로 지정할 필요 없음
    ## output을 참고하여 차원을 계산해주세요!
    self.max_pool1 = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=1, kernel_size=2, padding=1),
        nn.MaxPool2d(kernel_size=2)
    )
    self.max_pool2 = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=1, kernel_size=2, padding=1),
        nn.MaxPool2d(kernel_size=2)
    )
    self.fc1 = nn.Linear(in_features=10*5*5, out_features=50)
    self.fc2 = nn.Linear(in_features=50, out_features=10)

  def forward(self, x):
    print("연산 전", x.size())
    x = F.relu(self.max_pool1(x))
    print("max_pool1 연산 후", x.size())
    x = F.relu(self.max_pool2(x))
    print("max_pool2 연산 후",x.size())
    x = x.view(-1, 10 * 5 * 5)
    print("차원 감소 후", x.size())
    x = F.relu(self.fc1(x))
    print("fc1 연산 후", x.size())
    x = self.fc2(x)
    print("fc2 연산 후", x.size())
    return x

cnn = CNN()
output = cnn(torch.randn(10, 1, 20, 20))

연산 전 torch.Size([10, 1, 20, 20])
max_pool1 연산 후 torch.Size([10, 1, 10, 10])
max_pool2 연산 후 torch.Size([10, 1, 5, 5])
차원 감소 후 torch.Size([1, 250])
fc1 연산 후 torch.Size([1, 50])
fc2 연산 후 torch.Size([1, 10])


## MNIST 데이터셋 train

In [7]:
!pip install torchvision

In [8]:
import torch.optim as optim
from torchvision import datasets, transforms

In [18]:
train_data = datasets.MNIST('./data/', train=True, download=True, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])) # 학습 데이터
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=50, shuffle=True)

test_data = datasets.MNIST('./data/', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])) # 테스트 데이터
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=50, shuffle=True)

In [19]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        ## 조건1: conv layer 2개 & fc layer 2개
            ## 첫 번째 conv layer: 입력 채널 1, 출력 채널 20
            ## 두 번째 fc layer: 출력 채널 10
        ## 조건2: kernel_size=5, stride=1
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=50, kernel_size=5, stride=1)
        self.fc1 = nn.Linear(in_features=50*4*4, out_features=50)
        self.fc2 = nn.Linear(in_features=50, out_features=10)

    def forward(self, x):

        ## 조건1: relu -> max_pool2d -> relu -> max_pool2d 순서로 이루어짐
        ## 조건2: kernel_size=2, stride=2
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        x = x.view(-1, 4 * 4 * 50) # [batch_size, 50, 4, 4]
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x

In [20]:
cnn = CNN()
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr=0.01)

In [21]:
cnn.train()  # 학습을 위함
for epoch in range(10):
  for index, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()  # 기울기 초기화
    output = cnn(data)
    loss = criterion(output, target)
    loss.backward()  # 역전파
    optimizer.step()

    if index % 100 == 0:
      print("loss of {} epoch, {} index : {}".format(epoch, index, loss.item()))

loss of 0 epoch, 0 index : 2.3030405044555664
loss of 0 epoch, 100 index : 1.5961107015609741
loss of 0 epoch, 200 index : 0.5063483119010925
loss of 0 epoch, 300 index : 0.2967080771923065
loss of 0 epoch, 400 index : 0.2291674166917801
loss of 0 epoch, 500 index : 0.2898487150669098
loss of 0 epoch, 600 index : 0.31588876247406006
loss of 0 epoch, 700 index : 0.34736186265945435
loss of 0 epoch, 800 index : 0.07123079150915146
loss of 0 epoch, 900 index : 0.08498483896255493
loss of 0 epoch, 1000 index : 0.33528533577919006
loss of 0 epoch, 1100 index : 0.30079174041748047
loss of 1 epoch, 0 index : 0.3435586988925934
loss of 1 epoch, 100 index : 0.306460440158844
loss of 1 epoch, 200 index : 0.05701503902673721
loss of 1 epoch, 300 index : 0.09925548732280731
loss of 1 epoch, 400 index : 0.13906477391719818
loss of 1 epoch, 500 index : 0.08920513093471527
loss of 1 epoch, 600 index : 0.18020641803741455
loss of 1 epoch, 700 index : 0.13429851830005646
loss of 1 epoch, 800 index : 0.

In [22]:
cnn.eval()  # test case 학습 방지를 위함
test_loss = 0
correct = 0
with torch.no_grad():
  for data, target in test_loader:
    output = cnn(data)
    test_loss += criterion(output, target).item() # sum up batch loss
    pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


Test set: Average loss: 7.1055, Accuracy: 9873/10000 (99%)

